# Config Tuning: Bolinas Surf Forecast

Load CDIP MOP MA147 hindcast data and compare how config parameter changes
affect swell scores and surf height predictions across a range of real conditions.

**Workflow:**
1. Load hindcast and baseline config
2. Auto-select representative days (big/small swell, clean/messy, long/short period)
3. Compute baseline scores
4. Modify config values in the Experiment cell
5. Compare via table and plots

In [1]:
import sys
from pathlib import Path
import copy
import yaml
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt

ROOT = Path.cwd().parent if Path.cwd().name == "simulations" else Path.cwd()
sys.path.insert(0, str(ROOT / "src"))

from process_data import compute_swell_score, predict_bolinas_surf_height

## 1. Load Hindcast Data

Pulls MA147 hindcast from CDIP THREDDS, filtered to the swell frequency band.

In [2]:
HINDCAST_URL = (
    "https://thredds.cdip.ucsd.edu/thredds/dodsC/"
    "cdip/model/MOP_alongshore/MA147_hindcast.nc"
)

hc = xr.open_dataset(HINDCAST_URL, engine="netcdf4")
hc_swell = hc.sel(waveFrequency=hc.waveFrequency[hc.waveFrequency <= 0.12]).load()

print(f"Loaded {len(hc_swell.waveTime)} timesteps")
print(f"Time range: {pd.Timestamp(hc_swell.waveTime.values[0])} to {pd.Timestamp(hc_swell.waveTime.values[-1])}")

Loaded 221328 timesteps
Time range: 2000-01-01 00:00:00 to 2025-03-31 23:00:00


In [3]:
NOWCAST_URL = (
    "https://thredds.cdip.ucsd.edu/thredds/dodsC/"
    "cdip/model/MOP_alongshore/MA147_nowcast.nc"
)

nc = xr.open_dataset(NOWCAST_URL, engine="netcdf4")
nc_swell = nc.sel(waveFrequency=nc.waveFrequency[nc.waveFrequency <= 0.12]).load()

print(f"Loaded {len(nc_swell.waveTime)} timesteps")
print(f"Time range: {pd.Timestamp(nc_swell.waveTime.values[0])} to {pd.Timestamp(nc_swell.waveTime.values[-1])}")
print(nc_swell.dims)

Loaded 7702 timesteps
Time range: 2025-04-01 00:00:00 to 2026-02-15 21:00:00
FrozenMappingWarningOnValuesAccess({'waveTime': 7702, 'metaBoundsCount': 2, 'waveFrequency': 12})


In [4]:
ds_swell = xr.concat([hc_swell, nc_swell], dim='waveTime')

## 2. Load Baseline Config

In [5]:
with open(ROOT / "config" / "surf_config.yaml", "r") as f:
    baseline_config = yaml.safe_load(f)

baseline_surf = baseline_config["surf_model"]
print("Spectral scoring:", baseline_surf["spectral_scoring"])
print("Nearshore:", baseline_surf["nearshore"])

Spectral scoring: {'hs_min_m': 0.3, 'hs_full_credit_m': 1.5, 'tp_min_s': 10.0, 'tp_full_credit_s': 16.0, 'spread_min_deg': 5.0, 'spread_max_deg': 20.0, 'w_hs': 0.3, 'w_tp': 0.5, 'w_sp': 0.2}
Nearshore: {'range_factor': 0.15, 'range_period_min': 12, 'range_step': 0.01, 'dp_neutral': 215.0, 'dp_slope': 0.04, 'dp_max_boost': 2.0, 'dp_min_factor': 0.4}


## 3. Compute Baseline Scores

In [6]:
def score_with_config(ds, surf_cfg):
    """Compute swell scores and surf heights for a given config."""
    scores = compute_swell_score(ds, surf_cfg)
    heights = scores.apply(
        lambda row: predict_bolinas_surf_height(
            row["hs_swell"] * 3.28084, row["tp"], row["dp"], surf_cfg["nearshore"]
        ),
        axis=1,
    )
    return pd.concat([scores, pd.DataFrame(heights.tolist(), index=scores.index)], axis=1)

baseline_scores = score_with_config(ds_swell, baseline_surf)
baseline_scores[["swell_score", "hs_swell", "tp", "dp", "spread", "bolinas_surf_min_ft", "bolinas_surf_max_ft"]].head(10)

,swell_score,hs_swell,tp,dp,spread,bolinas_surf_min_ft,bolinas_surf_max_ft
time,,,,,,,
2000-01-01 00:00:00,1.986188,0.375544,10.000000,226.670746,17.427141,0.5,1.0
2000-01-01 01:00:00,1.958619,0.369147,10.000000,227.572632,17.412550,0.5,0.5
2000-01-01 02:00:00,1.883554,0.361445,10.000000,227.557068,17.728409,0.5,0.5
2000-01-01 03:00:00,1.797844,0.355029,10.000000,226.943298,18.169527,0.5,0.5
2000-01-01 04:00:00,1.794327,0.353304,9.090909,226.380615,18.122740,0.5,0.5
2000-01-01 05:00:00,2.042689,0.364203,9.090909,233.992264,16.515305,0.5,0.5
2000-01-01 06:00:00,2.412758,0.399935,9.090909,236.706970,14.720095,0.5,0.5
2000-01-01 07:00:00,2.601640,0.421252,9.090909,236.737091,13.805140,0.5,0.5
2000-01-01 08:00:00,2.650622,0.432497,9.090909,236.646210,13.721259,0.5,0.5


## 4. Select Representative Days

Auto-selects days from the hindcast that cover a range of conditions:
- Biggest swell day
- Smallest swell day (still active)
- Longest period day
- Shortest period day
- Cleanest swell (narrowest spread)
- Messiest swell (widest spread)
- Highest scoring day
- Lowest scoring day

In [7]:
# Compute daily summaries from baseline
daily = baseline_scores.copy()
daily["date"] = daily.index.date

daily_agg_cols = {
    "avg_score": ("swell_score", "mean"),
    "max_hs": ("hs_swell", "max"),
    "avg_tp": ("tp", "mean"),
    "avg_dp": ("dp", "mean"),
    "avg_spread": ("spread", "mean"),
    "avg_hs_score": ("hs_score", "mean"),
    "avg_tp_score": ("tp_score", "mean"),
    "avg_spread_score": ("spread_score", "mean"),
    "avg_surf_min": ("bolinas_surf_min_ft", "mean"),
    "avg_surf_max": ("bolinas_surf_max_ft", "mean"),
}
if "dp_score" in daily.columns:
    daily_agg_cols["avg_dp_score"] = ("dp_score", "mean")
daily_agg = daily.groupby("date").agg(**daily_agg_cols)

# Filter out flat days (no swell at all)
active = daily_agg[daily_agg["max_hs"] > 0.1]

# Pick representative days across the full range of conditions
picks = {}

# Extremes
picks["Biggest swell"] = active["max_hs"].idxmax()
picks["Smallest swell"] = active["max_hs"].idxmin()
picks["Longest period"] = active["avg_tp"].idxmax()
picks["Shortest period"] = active["avg_tp"].idxmin()
picks["Cleanest (narrow spread)"] = active["avg_spread"].idxmin()
picks["Messiest (wide spread)"] = active["avg_spread"].idxmax()
picks["Highest score"] = active["avg_score"].idxmax()
picks["Lowest score"] = active["avg_score"].idxmin()
picks["Most Southerly"] = active["avg_dp"].idxmin()
picks["Most Northerly"] = active["avg_dp"].idxmin()


# Average / typical days (closest to median for each metric)
def nearest_to_median(series):
    med = series.median()
    return (series - med).abs().idxmin()

picks["Avg swell size"] = nearest_to_median(active["max_hs"])
picks["Avg period"] = nearest_to_median(active["avg_tp"])
picks["Avg direction"] = nearest_to_median(active["avg_dp"])
picks["Avg score"] = nearest_to_median(active["avg_score"])
picks["Avg spread"] = nearest_to_median(active["avg_spread"])

# Deduplicate (same day might win multiple categories)
seen = set()
unique_picks = {}
for label, date in picks.items():
    if date not in seen:
        unique_picks[label] = date
        seen.add(date)

rep_days = daily_agg.loc[list(unique_picks.values())].copy()
rep_days.insert(0, "condition", list(unique_picks.keys()))
rep_days = rep_days.sort_values("avg_score", ascending=False)

print(f"Selected {len(rep_days)} representative days:\n")
rep_days


Selected 13 representative days:



,condition,avg_score,max_hs,avg_tp,avg_dp,avg_spread,avg_hs_score,avg_tp_score,avg_spread_score,avg_surf_min,avg_surf_max
date,,,,,,,,,,,
2003-01-07,Cleanest (narrow spread),9.850831,3.574147,19.545454,216.725586,6.243074,1.000000,1.000000,0.917128,6.750000,10.583333
2020-12-03,Longest period,9.283855,1.778847,21.643518,214.152954,8.013818,0.868709,1.000000,0.799079,3.229167,5.312500
2003-02-09,Avg direction,8.972603,1.349155,17.317406,215.500046,7.191408,0.724194,0.995611,0.853906,2.354167,3.625000
2023-01-05,Biggest swell,8.104911,6.418896,14.840068,206.961121,8.415290,0.994930,0.672986,0.772314,12.375000,18.250000
2020-06-01,Avg swell size,7.054368,0.786264,13.882153,197.611618,12.211991,0.560606,0.801371,0.519201,3.125000,4.416667
2008-09-21,Avg score,6.463272,0.747455,13.890693,200.982773,13.632820,0.449240,0.774725,0.424479,2.375000,3.375000
2013-05-06,Most Southerly,5.473900,0.361596,13.769841,188.133926,14.434258,0.091139,0.791097,0.371049,1.666667,2.479167
2001-05-19,Avg period,5.423306,1.004154,13.333333,223.683777,12.005025,0.727374,0.333333,0.532998,1.875000,2.958333
2024-02-14,Avg spread,4.856475,0.589423,11.808196,217.794067,12.442201,0.454810,0.382567,0.503853,1.312500,1.812500


## 5. Experiment: Modify Config

Edit the values below, then re-run this cell and the cells that follow.

Common things to try:
- `hs_min_m`: raise to penalize small swells more (default 0.3)
- `hs_full_credit_m`: lower to give full marks to smaller waves (default 1.5)
- `tp_min_s` / `tp_full_credit_s`: shift the period scoring curve
- `spread_max_deg`: raise to be more tolerant of messy swells (default 20)
- `w_hs` / `w_tp` / `w_sp`: rebalance the scoring weights (must sum to 1.0)
- `range_factor`: widen/narrow the surf height min-max spread (default 0.15)

In [15]:
experiment_config = copy.deepcopy(baseline_surf)

# --- EDIT BELOW ---
experiment_config["spectral_scoring"]["hs_min_m"] = 0.3
experiment_config["spectral_scoring"]["hs_full_credit_m"] = 1.5
experiment_config["spectral_scoring"]["tp_min_s"] = 10.0
experiment_config["spectral_scoring"]["tp_full_credit_s"] = 16.0
experiment_config["spectral_scoring"]["spread_min_deg"] = 5.0
experiment_config["spectral_scoring"]["spread_max_deg"] = 20.0
experiment_config["spectral_scoring"]["w_hs"] = 0.30
experiment_config["spectral_scoring"]["w_tp"] = 0.50
experiment_config["spectral_scoring"]["w_sp"] = 0.20

experiment_config["nearshore"]["range_factor"] = 0.15
experiment_config["nearshore"]["dp_neutral"] = 215
experiment_config["nearshore"]["dp_slope"] = 0.04
experiment_config["nearshore"]["dp_max_boost"] = 2.00
experiment_config["nearshore"]["dp_min_factor"] = 0.8
# --- EDIT ABOVE ---

experiment_scores = score_with_config(ds_swell, experiment_config)

print(f"Baseline avg swell score:    {baseline_scores['swell_score'].mean():.2f}")
print(f"Experiment avg swell score:  {experiment_scores['swell_score'].mean():.2f}")
print(f"Delta:                       {experiment_scores['swell_score'].mean() - baseline_scores['swell_score'].mean():+.2f}")

Baseline avg swell score:    6.18
Experiment avg swell score:  6.18
Delta:                       +0.00


## 6. Day-by-Day Comparison Table

Shows how each representative day changes between baseline and experiment config.

In [20]:
# Aggregate experiment scores by day
exp_daily = experiment_scores.copy()
exp_daily["date"] = exp_daily.index.date

exp_agg_cols = {
    "avg_score": ("swell_score", "mean"),
    "max_hs": ("hs_swell", "max"),
    "avg_tp": ("tp", "mean"),
    "avg_dp": ("dp", "mean"),
    "avg_spread": ("spread", "mean"),
    "avg_hs_score": ("hs_score", "mean"),
    "avg_tp_score": ("tp_score", "mean"),
    "avg_spread_score": ("spread_score", "mean"),
    "avg_surf_min": ("bolinas_surf_min_ft", "mean"),
    "avg_surf_max": ("bolinas_surf_max_ft", "mean"),
}
if "dp_score" in exp_daily.columns:
    exp_agg_cols["avg_dp_score"] = ("dp_score", "mean")
exp_daily_agg = exp_daily.groupby("date").agg(**exp_agg_cols)

# --- Manually reviewed days ---
manual_days = pd.to_datetime(['2023-05-12', '2023-09-03', '2023-10-19', '2023-10-27',
                              '2024-05-12', '2024-07-05', '2025-03-01', '2025-06-03',
                              '2025-08-11', '2025-11-14', '2026-01-29', '2026-02-02',
                              '2026-02-07'])
manual_labels = [
    'Manual: 7 score',
    'Manual: 9 score, overhead',
    'Manual: good swell, high tide challenged',
    'Manual: 6 score, stomach-chest',
    'Manual: 8 score, shoulder-head',
    'Manual: 7 score, shoulder',
    'Manual: 8 score, shoulder-head',
    'Manual: 8 score, chest-shoulder',
    'Manual: 8 score, shoulder at best',
    'Manual: 8 score, little less than head',
    'Manual: 6-7 score lully and wind, 3-4ft',
    'Manual: 9 score, 3-6ft',
    'Manual: 9-7 score (dropping throughout due to tide), 3-6ft',
]

manual_dates = [d.date() for d in manual_days]
manual_in_data = [d for d in manual_dates if d in daily_agg.index]
manual_labels_in_data = [l for d, l in zip(manual_dates, manual_labels) if d in daily_agg.index]

# Combine auto-selected + manual days (deduplicated)
rep_dates = rep_days.index.tolist()
rep_conditions = rep_days["condition"].tolist()

all_dates = []
all_conditions = []
seen_dates = set()

for date, cond in zip(rep_dates, rep_conditions):
    all_dates.append(date)
    all_conditions.append(cond)
    seen_dates.add(date)

for date, cond in zip(manual_in_data, manual_labels_in_data):
    if date not in seen_dates:
        all_dates.append(date)
        all_conditions.append(cond)
        seen_dates.add(date)

base_all = daily_agg.loc[all_dates].copy()
exp_all = exp_daily_agg.loc[all_dates].copy()

# Build score comparison with components
comparison_data = {
    "Condition": all_conditions,
    "Date": all_dates,
    "Hs (m)": base_all["max_hs"].round(2).values,
    "Tp (s)": base_all["avg_tp"].round(1).values,
    "Dp (deg)": base_all["avg_dp"].round(1).values,
    "Spread": base_all["avg_spread"].round(1).values,
    # "B hs_sc": base_all["avg_hs_score"].round(2).values,
    # "E hs_sc": exp_all["avg_hs_score"].round(2).values,
    # "B tp_sc": base_all["avg_tp_score"].round(2).values,
    # "E tp_sc": exp_all["avg_tp_score"].round(2).values,
    # "B sp_sc": base_all["avg_spread_score"].round(2).values,
    # "E sp_sc": exp_all["avg_spread_score"].round(2).values,
}

comparison_data.update({
    "Baseline Score": base_all["avg_score"].round(1).values,
    "Experiment Score": exp_all["avg_score"].round(1).values,
    "Score Delta": (exp_all["avg_score"].values - base_all["avg_score"].values).round(1),
})

comparison = pd.DataFrame(comparison_data)

def highlight_delta(val):
    if val > 0:
        return "color: green; font-weight: bold"
    elif val < 0:
        return "color: red; font-weight: bold"
    return ""

comparison.style.applymap(highlight_delta, subset=["Score Delta"])


/var/folders/5v/cz1w61mx7hg5nn2hy8ybbf3c0000gn/T/ipykernel_12025/2538219757.py:99: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  comparison.style.applymap(highlight_delta, subset=["Score Delta"])


,Condition,Date,Hs (m),Tp (s),Dp (deg),Spread,Baseline Score,Experiment Score,Score Delta
0,Cleanest (narrow spread),2003-01-07,3.570000,19.500000,216.699997,6.200000,9.900000,9.900000,0.000000
1,Longest period,2020-12-03,1.780000,21.600000,214.199997,8.000000,9.300000,9.300000,0.000000
2,Avg direction,2003-02-09,1.350000,17.299999,215.500000,7.200000,9.000000,9.000000,0.000000
3,Biggest swell,2023-01-05,6.420000,14.800000,207.000000,8.400000,8.100000,8.100000,0.000000
4,Avg swell size,2020-06-01,0.790000,13.900000,197.600006,12.200000,7.100000,7.100000,0.000000
5,Avg score,2008-09-21,0.750000,13.900000,201.000000,13.600000,6.500000,6.500000,0.000000
6,Most Southerly,2013-05-06,0.360000,13.800000,188.100006,14.400000,5.500000,5.500000,0.000000
7,Avg period,2001-05-19,1.000000,13.300000,223.699997,12.000000,5.400000,5.400000,0.000000
8,Avg spread,2024-02-14,0.590000,11.800000,217.800003,12.400000,4.900000,4.900000,0.000000
9,Messiest (wide spread),2008-01-28,1.280000,9.900000,212.199997,20.299999,3.300000,3.300000,0.000000


## 7. Surf Height Comparison Table

Shows how predicted surf heights change between baseline and experiment config
for all representative + manually reviewed days.

In [21]:
height_comparison = pd.DataFrame({
    "Condition": all_conditions,
    "Date": all_dates,
    "Hs (m)": base_all["max_hs"].round(2).values,
    "Tp (s)": base_all["avg_tp"].round(1).values,
    "Dp (deg)": base_all["avg_dp"].round(1).values,
    "B Min (ft)": base_all["avg_surf_min"].round(1).values,
    "B Max (ft)": base_all["avg_surf_max"].round(1).values,
    "E Min (ft)": exp_all["avg_surf_min"].round(1).values,
    "E Max (ft)": exp_all["avg_surf_max"].round(1).values,
    "Min Delta": (exp_all["avg_surf_min"].values - base_all["avg_surf_min"].values).round(1),
    "Max Delta": (exp_all["avg_surf_max"].values - base_all["avg_surf_max"].values).round(1),
})

def highlight_height_delta(val):
    if val > 0.5:
        return "color: green; font-weight: bold"
    elif val < -0.5:
        return "color: red; font-weight: bold"
    elif val > 0:
        return "color: green"
    elif val < 0:
        return "color: red"
    return ""

height_comparison.style.applymap(highlight_height_delta, subset=["Min Delta", "Max Delta"])

/var/folders/5v/cz1w61mx7hg5nn2hy8ybbf3c0000gn/T/ipykernel_12025/2932740461.py:26: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  height_comparison.style.applymap(highlight_height_delta, subset=["Min Delta", "Max Delta"])


,Condition,Date,Hs (m),Tp (s),Dp (deg),B Min (ft),B Max (ft),E Min (ft),E Max (ft),Min Delta,Max Delta
0,Cleanest (narrow spread),2003-01-07,3.570000,19.500000,216.699997,6.800000,10.600000,6.800000,10.600000,0.000000,0.000000
1,Longest period,2020-12-03,1.780000,21.600000,214.199997,3.200000,5.300000,3.200000,5.300000,0.000000,0.000000
2,Avg direction,2003-02-09,1.350000,17.299999,215.500000,2.400000,3.600000,2.400000,3.600000,0.000000,0.000000
3,Biggest swell,2023-01-05,6.420000,14.800000,207.000000,12.400000,18.200000,12.400000,18.200000,0.000000,0.000000
4,Avg swell size,2020-06-01,0.790000,13.900000,197.600006,3.100000,4.400000,3.100000,4.400000,0.000000,0.000000
5,Avg score,2008-09-21,0.750000,13.900000,201.000000,2.400000,3.400000,2.400000,3.400000,0.000000,0.000000
6,Most Southerly,2013-05-06,0.360000,13.800000,188.100006,1.700000,2.500000,1.700000,2.500000,0.000000,0.000000
7,Avg period,2001-05-19,1.000000,13.300000,223.699997,1.900000,3.000000,2.500000,3.900000,0.700000,0.900000
8,Avg spread,2024-02-14,0.590000,11.800000,217.800003,1.300000,1.800000,1.600000,2.100000,0.300000,0.300000
9,Messiest (wide spread),2008-01-28,1.280000,9.900000,212.199997,3.400000,4.600000,3.500000,4.800000,0.100000,0.100000


---

## 9. Wind & Tide Config Tuning

Test how wind and tide config parameters affect the **final surf score**.
Uses `calculate_surf_score()` which combines swell, wind, and tide into a single 1-10 score.

We simulate a grid of conditions: a few fixed swell scores paired with
varying wind speeds, wind categories, and tide heights.

In [11]:
from process_data import calculate_surf_score

# Fixed swell scenarios to test against
swell_scenarios = {
    "Small clean swell (score=4)": 4.0,
    "Medium swell (score=6)": 6.0,
    "Pumping swell (score=8)": 8.0,
}

# Wind conditions to sweep
wind_conditions = [
    {"label": "Glassy (3 mph offshore)", "speed": 3.0, "gust": 5.0, "cat": "offshore"},
    {"label": "Light offshore (8 mph)", "speed": 8.0, "gust": 12.0, "cat": "offshore"},
    {"label": "Moderate cross (10 mph)", "speed": 10.0, "gust": 14.0, "cat": "crosshore"},
    {"label": "Moderate onshore (12 mph)", "speed": 12.0, "gust": 18.0, "cat": "onshore"},
    {"label": "Strong onshore (18 mph)", "speed": 18.0, "gust": 25.0, "cat": "onshore"},
]

# Tide conditions to sweep
tide_heights = [-0.5, 0.5, 1.0, 2.0, 3.5, 5.0]


### 9a. Wind Config Experiment

Edit the wind config values below, then re-run to see how the final surf score changes.

In [12]:
wind_exp_config = copy.deepcopy(baseline_surf)

# --- EDIT WIND CONFIG BELOW ---
wind_exp_config["wind"]["speed_floor"] = 5.0          # mph below this = no penalty
wind_exp_config["wind"]["speed_range"] = 13.0         # mph from glass to blown out
wind_exp_config["wind"]["gust_weight"] = 0.3          # 0=ignore gusts, 1=gusts only
wind_exp_config["wind"]["offshore_penalty_weight"] = 0.7
wind_exp_config["wind"]["crosshore_penalty_weight"] = 1.0
wind_exp_config["wind"]["onshore_penalty_weight"] = 1.0
wind_exp_config["wind"]["penalty_min"] = 0.25         # floor multiplier (blown out)
wind_exp_config["final_scoring"]["wind_impact_weight"] = 0.60
# --- EDIT WIND CONFIG ABOVE ---

# Build comparison: baseline vs experiment across all wind/swell combos
rows = []
for swell_label, swell_score in swell_scenarios.items():
    for w in wind_conditions:
        base_final, _, base_wind, _ = calculate_surf_score(
            swell_score, w["speed"], w["gust"], w["cat"], 1.0, baseline_surf
        )
        exp_final, _, exp_wind, _ = calculate_surf_score(
            swell_score, w["speed"], w["gust"], w["cat"], 1.0, wind_exp_config
        )
        rows.append({
            "Swell": swell_label,
            "Wind": w["label"],
            "Baseline Wind Score": base_wind,
            "Exp Wind Score": exp_wind,
            "Baseline Final": base_final,
            "Exp Final": exp_final,
            "Delta": round(exp_final - base_final, 1),
        })

wind_comparison = pd.DataFrame(rows)
wind_comparison.style.applymap(highlight_delta, subset=["Delta"])


/var/folders/5v/cz1w61mx7hg5nn2hy8ybbf3c0000gn/T/ipykernel_12025/2291957163.py:35: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  wind_comparison.style.applymap(highlight_delta, subset=["Delta"])


,Swell,Wind,Baseline Wind Score,Exp Wind Score,Baseline Final,Exp Final,Delta
0,Small clean swell (score=4),Glassy (3 mph offshore),10.000000,10.000000,4.000000,4.000000,0.000000
1,Small clean swell (score=4),Light offshore (8 mph),8.900000,8.900000,4.000000,4.000000,0.000000
2,Small clean swell (score=4),Moderate cross (10 mph),5.200000,5.200000,4.000000,4.000000,0.000000
3,Small clean swell (score=4),Moderate onshore (12 mph),3.200000,3.200000,3.200000,3.200000,0.000000
4,Small clean swell (score=4),Strong onshore (18 mph),2.500000,2.500000,2.500000,2.500000,0.000000
5,Medium swell (score=6),Glassy (3 mph offshore),10.000000,10.000000,6.000000,6.000000,0.000000
6,Medium swell (score=6),Light offshore (8 mph),8.900000,8.900000,6.000000,6.000000,0.000000
7,Medium swell (score=6),Moderate cross (10 mph),5.200000,5.200000,5.200000,5.200000,0.000000
8,Medium swell (score=6),Moderate onshore (12 mph),3.200000,3.200000,3.600000,3.600000,0.000000
9,Medium swell (score=6),Strong onshore (18 mph),2.500000,2.500000,3.300000,3.300000,0.000000


### 9b. Tide Config Experiment

Edit the tide config values below, then re-run to see how the final surf score changes.

In [13]:
tide_exp_config = copy.deepcopy(baseline_surf)

# --- EDIT TIDE CONFIG BELOW ---
tide_exp_config["tide"]["optimal_height"] = 1.0       # center of Gaussian (ft MLLW)
tide_exp_config["tide"]["sigma"] = 2.5                # width: larger = more tolerant
tide_exp_config["tide"]["penalty_min"] = 0.1          # floor multiplier
tide_exp_config["final_scoring"]["tide_impact_weight"] = 0.60
# --- EDIT TIDE CONFIG ABOVE ---

# Build comparison: baseline vs experiment across all tide/swell combos
rows = []
for swell_label, swell_score in swell_scenarios.items():
    for tide in tide_heights:
        base_final, _, _, base_tide = calculate_surf_score(
            swell_score, 3.0, 5.0, "offshore", tide, baseline_surf
        )
        exp_final, _, _, exp_tide = calculate_surf_score(
            swell_score, 3.0, 5.0, "offshore", tide, tide_exp_config
        )
        rows.append({
            "Swell": swell_label,
            "Tide (ft)": tide,
            "Baseline Tide Score": base_tide,
            "Exp Tide Score": exp_tide,
            "Baseline Final": base_final,
            "Exp Final": exp_final,
            "Delta": round(exp_final - base_final, 1),
        })

tide_comparison = pd.DataFrame(rows)
tide_comparison.style.applymap(highlight_delta, subset=["Delta"])


/var/folders/5v/cz1w61mx7hg5nn2hy8ybbf3c0000gn/T/ipykernel_12025/1479117211.py:31: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  tide_comparison.style.applymap(highlight_delta, subset=["Delta"])


,Swell,Tide (ft),Baseline Tide Score,Exp Tide Score,Baseline Final,Exp Final,Delta
0,Small clean swell (score=4),-0.500000,8.400000,8.400000,4.000000,4.000000,0.000000
1,Small clean swell (score=4),0.500000,9.800000,9.800000,4.000000,4.000000,0.000000
2,Small clean swell (score=4),1.000000,10.000000,10.000000,4.000000,4.000000,0.000000
3,Small clean swell (score=4),2.000000,9.200000,9.200000,4.000000,4.000000,0.000000
4,Small clean swell (score=4),3.500000,6.100000,6.100000,4.000000,4.000000,0.000000
5,Small clean swell (score=4),5.000000,2.800000,2.800000,2.800000,2.800000,0.000000
6,Medium swell (score=6),-0.500000,8.400000,8.400000,6.000000,6.000000,0.000000
7,Medium swell (score=6),0.500000,9.800000,9.800000,6.000000,6.000000,0.000000
8,Medium swell (score=6),1.000000,10.000000,10.000000,6.000000,6.000000,0.000000
9,Medium swell (score=6),2.000000,9.200000,9.200000,6.000000,6.000000,0.000000


### 9c. Combined: Full Scenario Grid

Applies both wind and tide experiment configs simultaneously to see the combined effect.

In [14]:
combined_config = copy.deepcopy(baseline_surf)
combined_config["wind"] = copy.deepcopy(wind_exp_config["wind"])
combined_config["tide"] = copy.deepcopy(tide_exp_config["tide"])
combined_config["final_scoring"] = copy.deepcopy(wind_exp_config["final_scoring"])
combined_config["final_scoring"]["tide_impact_weight"] = tide_exp_config["final_scoring"]["tide_impact_weight"]

# Pick a few representative combos
combo_winds = [wind_conditions[0], wind_conditions[2], wind_conditions[4]]  # glassy, mod cross, strong onshore
combo_tides = [-0.5, 1.0, 3.5]

rows = []
for swell_label, swell_score in swell_scenarios.items():
    for w in combo_winds:
        for tide in combo_tides:
            base_final, _, _, _ = calculate_surf_score(
                swell_score, w["speed"], w["gust"], w["cat"], tide, baseline_surf
            )
            exp_final, _, _, _ = calculate_surf_score(
                swell_score, w["speed"], w["gust"], w["cat"], tide, combined_config
            )
            rows.append({
                "Swell": swell_label,
                "Wind": w["label"],
                "Tide (ft)": tide,
                "Baseline Final": base_final,
                "Exp Final": exp_final,
                "Delta": round(exp_final - base_final, 1),
            })

combined_comparison = pd.DataFrame(rows)
combined_comparison.style.applymap(highlight_delta, subset=["Delta"])


/var/folders/5v/cz1w61mx7hg5nn2hy8ybbf3c0000gn/T/ipykernel_12025/851106371.py:31: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  combined_comparison.style.applymap(highlight_delta, subset=["Delta"])


,Swell,Wind,Tide (ft),Baseline Final,Exp Final,Delta
0,Small clean swell (score=4),Glassy (3 mph offshore),-0.500000,4.000000,4.000000,0.000000
1,Small clean swell (score=4),Glassy (3 mph offshore),1.000000,4.000000,4.000000,0.000000
2,Small clean swell (score=4),Glassy (3 mph offshore),3.500000,4.000000,4.000000,0.000000
3,Small clean swell (score=4),Moderate cross (10 mph),-0.500000,4.000000,4.000000,0.000000
4,Small clean swell (score=4),Moderate cross (10 mph),1.000000,4.000000,4.000000,0.000000
5,Small clean swell (score=4),Moderate cross (10 mph),3.500000,4.000000,4.000000,0.000000
6,Small clean swell (score=4),Strong onshore (18 mph),-0.500000,2.500000,2.500000,0.000000
7,Small clean swell (score=4),Strong onshore (18 mph),1.000000,2.500000,2.500000,0.000000
8,Small clean swell (score=4),Strong onshore (18 mph),3.500000,2.500000,2.500000,0.000000
9,Medium swell (score=6),Glassy (3 mph offshore),-0.500000,6.000000,6.000000,0.000000
